In [ ]:
import itertools

import openai
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
# connect to server for hypothesis generation
import sys
import time


port = 80

server_ip = sys.argv[1]

# Set OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "cmsc-35360"
openai_api_base = f"http://195.88.24.64:{port}/v1"


prompt = (
    "Read the following scientific step-by-step protocol carefully, and generate synthetic data simulating the outcomes of the experiment."
    " Use the materials generated in the following protocol and simulating run several tests, varying the amounts of each material and attempting to simulate the"
    " results of the protocol in a lab setting. I want a dataframe returned where each row is a different experimental configuration, with each column accounting for the"
    " amount of material used, with the last column recording the outcome of the experiment. Simulate 25 experiments, so there should be a dataframe returned with"
    " 25 rows and several columns, one for each material and one for the results of that simulated experiment. Attempt to give me data for what the experiment"
    " would result in based on a lab setting. Return only a dataframe and nothing else."
)

prompt2 = (
    "Remember, aim to Simulate 25 experiments, so there should be a dataframe returned with"
    " 25 rows and several columns, and nothing else. DO not return any other words or code, just the dataframe. Do not write 'Here is the simulated dataframe'"
    " just return the dataframe."
)


def create_prompt(chunk):
    gpt_user_prompt = prompt + chunk + prompt2
    return gpt_user_prompt


client = openai.OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)


gpt_assistant_prompt = "You are a super smart AI that knows about science. You follow directions and you are always truthful and concise in your reponses."


def query_llm(gpt_user_prompt):
    message = [
        {"role": "assistant", "content": gpt_assistant_prompt},
        {"role": "user", "content": gpt_user_prompt},
    ]
    temperature = 0.0
    frequency_penalty = 0.0
    max_retries = 3
    retry_count = 0

    while retry_count < max_retries:
        try:
            response = client.chat.completions.create(
                model="meta-llama/Meta-Llama-3-70B-Instruct",
                messages=message,
                temperature=temperature,
                frequency_penalty=frequency_penalty,
            )
            time.sleep(2)
            return response.choices[0].message.content
            break
        except:
            print("Hypo trying again port " + str(port))
            retry_count += 1
            time.sleep(2)
            if retry_count == max_retries:
                print("Maximum retries reached. Exiting the program.")
                exit
    time.sleep(2)


In [ ]:
df = pd.read_csv("specific_protocol.csv")
df["data"] = None
df


In [ ]:
i = 0
for h, p in zip(df["HYPOTHESIS"], df["specific_plan"]):
    print(i)
    gpt_p = create_prompt(p)
    response = query_llm(gpt_p)
    df.at[i, "data"] = response
    i += 1


In [ ]:
df.to_csv("generated_data.csv", index=False)
